In the "fitting.ipynb" notebook, we specified a desired fit tolerance and the fit would keep adding basis functions until it achieved the given fit tolerance. If the data has a noise level and we specify the noise level as the fit tolerance, we obtain good results.

Unfortunately, we may not know how noisy our data is. 
For this we can use a cross-validation study, see Cross_validation_fitting.ipynb  for an example.
Another alternative is to use Gaussian Process Regression (GPR), which can also handle noise in a robust manner.

A good reference for GPR is "Gaussian Processes for Machine Learning" by Rasmussen and Williams.
Amazon link:  http://a.co/fIrYk11

In [1]:
import pySurrogate as pySur
import numpy as np
fit = pySur.fit
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('always')

## Simple 1-d GPR fit example

**Note**: Do not commit this notebook with changes to test values or parameters. It is used as a regression test in test_GPR_fit.

Run this code with 

```python
np.random.seed(0)
```

to reproduce the regression test. Note that scikit-learn uses numpy's random number generator, so setting a seed here will set the generator's global state. Also note that numpy's random number generator [may not be thread-safe](https://stackoverflow.com/questions/7029993/differences-between-numpy-random-and-random-random-in-python).

In [2]:
#np.random.seed(0)

# Exact solution
def f(x):
    return x*np.sin(x)

# The data can be noisy
def apply_noise(data):
    noise_level = 0.1
    dy = noise_level * np.random.random(data.shape)
    noise = np.random.normal(0, dy)
    return data + noise

# Choose some data points - let's fit between 1 and 8:
xMin = 1
xMax = 8
nPts = 6
xVals = np.linspace(xMin, xMax, nPts)
yVals = apply_noise(f(xVals))

# Setup a function modeler
ff = pySur.FittedFunction('1d-polynomial-fitter')
for x, y in zip(xVals, yVals):
    ff.addKnownDataSet(np.array([x]), y)

In [3]:
# Setup GPR and do the fit
ff.setupFits(minVals=[xMin], maxVals=[xMax], fitType='GPR')
ff.fit()

# evaluate at many points:
xDense = np.linspace(xMin, xMax, nPts*100)
evals = np.array([ff([x]) for x in xDense])

# GPR also gives an estimate of the error
err_estimates = np.array([ff.evalGPRErrorFunc([x]) for x in xDense])

# plot the results:
plt.plot(xDense, f(xDense), 'k', lw=2., label='Actual function')
plt.plot(ff.xVals.T[0], ff.yVals, 'kx', label='Data points')
plt.plot(xDense, evals, 'r--', lw=2., label='Fit')
plt.fill(np.concatenate([xDense, xDense[::-1]]),
         np.concatenate([evals - 1.9600 * err_estimates,
                        (evals + 1.9600 * err_estimates)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')
plt.legend()
plt.show()

In [4]:
# The fit is decent and GPR error prediction is meaningful: 
# The error is least near the datapoints and high where the fit deviates
# from the actual function.

# Now save try saving the fit to disk
print(ff([2]))
ff.saveH5('Greatest_fit_in_all_the_land.h5')

1.8120949466398089


In [5]:
# You can reload the fit as follows
ff = pySur.FittedFunction('1d-polynomial-fitter')
ff.loadH5('Greatest_fit_in_all_the_land.h5')
print(ff([2]))

1.8120949466398089


/home/balzani57/pool/anaconda3/envs/Python27/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/balzani57/pool/anaconda3/envs/Python27/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/balzani57/pool/anaconda3/envs/Python27/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/balzani57/pool/anaconda3/envs/Python27/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/balzani57/pool/anaconda3/envs/Python27/lib/python2

In [6]:
# There are more options for more control. These go into the ff.setupFits() function.
# See doc string of pySur.fit_gpr.gpr_fit_wrapper() for more details.
print pySur.fit_gpr.GPR_DEFAULTS

{'optimizer': 'fmin_l_bfgs_b', 'nu_Matern': 1.5, 'constKernel_init': 1.0, 'fitType': 'GPR', 'constKernel_bounds': (0.01, 100.0), 'noiseKernel_init': 0.0001, 'maxVals': 1.0, 'optimizer_nTrials': 10, 'doLinearFitBeforeGPR': True, 'noiseKernel_bounds': (1e-07, 0.01), 'kernelType': 'RBF', 'minVals': 0.0}


In [7]:
help(pySur.fit_gpr.gpr_fit_wrapper)

Help on function gpr_fit_wrapper in module pySurrogate.fit_gpr:

gpr_fit_wrapper(xVals, yVals, name, **options)
    Wrapper for GPRFitter class.
    Does GPR fitting using sklearn.GaussianProcessRegressor.
    The kernel or covariance function is set as
        kernel = ConstantKernel * MainKernel + WhiteKernel
        Here ConstantKernel just rescales the magnitude, MainKernel is set using
        the 'kernelType' option and WhiteKernel accounts for noise.
        We optimize the hyperparameters of the kernel and get the maximum
        likelihood values.
    
        Initial guess for length scale for the MainKernel in each dimension is
        chosen as the range of the xVals in that dimension. For the bounds of the
        length scales, we allow from 1/100 times to 10 times the above length
        scales. Smaller and larger scales are not expected to be important or
        resolvable.
    
    inputs:
    -------
    
    xVals: A 2d array of domain values, with shape (nPoints, 